In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

## Added one more CSV below

In [2]:
# Read in Affinity, COVID-19, and GeoID Data
affinity_df = pd.read_csv('Data/Affinity-State-Daily.csv')
covid_df = pd.read_csv('Data/COVID_Cases-State-Daily.csv')
geoids_df = pd.read_csv('Data/GeoIDs-State.csv')
ui_df = pd.read_csv('Data/UI_Claims-State-Weekly.csv')

In [3]:
#affinity_dff.count()
#covid_df.count()
geoids_df.count()

statefips        51
statename        51
stateabbrev      51
state_pop2019    51
dtype: int64

In [4]:
# Create date column
affinity_df.dtypes

year                     int64
month                    int64
day                      int64
statefips                int64
spend_acf              float64
spend_aer              float64
spend_all              float64
spend_all_inchigh       object
spend_all_inclow        object
spend_all_incmiddle    float64
spend_apg              float64
spend_grf              float64
spend_hcs              float64
spend_tws              float64
dtype: object

In [5]:
affinity_date = pd.to_datetime(affinity_df[['year', 'month', 'day']])
affinity_date

0      2020-01-24
1      2020-01-24
2      2020-01-24
3      2020-01-24
4      2020-01-24
          ...    
7441   2020-06-17
7442   2020-06-17
7443   2020-06-17
7444   2020-06-17
7445   2020-06-17
Length: 7446, dtype: datetime64[ns]

In [6]:
affinity_df["date"] = affinity_date
affinity_df = affinity_df[ ['date'] + [ col for col in affinity_df.columns if col != 'date' ] ]
affinity_df

,date,year,month,day,statefips,spend_acf,spend_aer,spend_all,spend_all_inchigh,spend_all_inclow,spend_all_incmiddle,spend_apg,spend_grf,spend_hcs,spend_tws
0,2020-01-24,2020,1,24,1,-0.000859,0.03580,0.00634,0.0219,0.000657,0.00687,-0.02130,0.00718,0.057000,-0.01760
1,2020-01-24,2020,1,24,2,0.007130,0.04920,-0.02640,-0.0106,.,-0.06990,0.15900,-0.03270,-0.154000,-0.07230
2,2020-01-24,2020,1,24,4,-0.009150,0.04490,-0.01260,-0.00749,-0.00918,-0.01670,-0.01190,-0.01160,0.020700,-0.04610
3,2020-01-24,2020,1,24,5,0.024400,-0.05270,-0.00707,0.0048,-0.00166,-0.01080,0.00842,-0.02800,0.005860,0.00444
4,2020-01-24,2020,1,24,6,0.004130,-0.00378,0.00813,0.00719,0.0124,0.00905,-0.01150,0.01610,0.020300,-0.00473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7441,2020-06-17,2020,6,17,51,-0.380000,-0.44500,-0.09680,-0.148,0.00955,-0.07610,-0.06580,0.04160,-0.166000,-0.53700
7442,2020-06-17,2020,6,17,53,-0.452000,-0.44700,-0.13100,-0.17,-0.162,-0.08700,-0.24200,0.04320,-0.223000,-0.49600
7443,2020-06-17,2020,6,17,54,-0.243000,-0.55300,0.00479,-0.0685,0.00212,0.01260,0.24000,0.05490,0.072500,-0.36200
7444,2020-06-17,2020,6,17,55,-0.375000,-0.42500,-0.09170,-0.1,-0.123,-0.08550,-0.03900,0.06040,-0.166000,-0.44700


In [7]:
covid_date = pd.to_datetime(covid_df[['year', 'month', 'day']])
covid_date

0      2020-01-21
1      2020-01-21
2      2020-01-21
3      2020-01-21
4      2020-01-21
          ...    
8920   2020-07-13
8921   2020-07-13
8922   2020-07-13
8923   2020-07-13
8924   2020-07-13
Length: 8925, dtype: datetime64[ns]

In [8]:
# add created date column to covid_df
covid_df["date"] = covid_date
covid_df = covid_df[ ['date'] + [ col for col in covid_df.columns if col != 'date' ] ]
covid_df

,date,year,month,day,statefips,case_rate,new_case_rate
0,2020-01-21,2020,1,21,1,0.0,.
1,2020-01-21,2020,1,21,2,0.0,.
2,2020-01-21,2020,1,21,4,0.0,.
3,2020-01-21,2020,1,21,5,0.0,.
4,2020-01-21,2020,1,21,6,0.0,.
...,...,...,...,...,...,...,...
8920,2020-07-13,2020,7,13,51,839.0,9.27
8921,2020-07-13,2020,7,13,53,572.0,9.42
8922,2020-07-13,2020,7,13,54,241.0,6.94
8923,2020-07-13,2020,7,13,55,697.0,13


In [9]:
# Merge covid and affinity dataframes
merge1_df = pd.merge(covid_df, affinity_df, on=["date", "statefips"], how="left")
merge1_df

,date,year_x,month_x,day_x,statefips,case_rate,new_case_rate,year_y,month_y,day_y,spend_acf,spend_aer,spend_all,spend_all_inchigh,spend_all_inclow,spend_all_incmiddle,spend_apg,spend_grf,spend_hcs,spend_tws
0,2020-01-21,2020,1,21,1,0.0,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-21,2020,1,21,2,0.0,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-21,2020,1,21,4,0.0,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-21,2020,1,21,5,0.0,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-21,2020,1,21,6,0.0,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8920,2020-07-13,2020,7,13,51,839.0,9.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8921,2020-07-13,2020,7,13,53,572.0,9.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8922,2020-07-13,2020,7,13,54,241.0,6.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8923,2020-07-13,2020,7,13,55,697.0,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
merge1_df = merge1_df.dropna(how='any')
merge1_df

,date,year_x,month_x,day_x,statefips,case_rate,new_case_rate,year_y,month_y,day_y,spend_acf,spend_aer,spend_all,spend_all_inchigh,spend_all_inclow,spend_all_incmiddle,spend_apg,spend_grf,spend_hcs,spend_tws
153,2020-01-24,2020,1,24,1,0.0,.,2020.0,1.0,24.0,-0.000859,0.03580,0.00634,0.0219,0.000657,0.00687,-0.02130,0.00718,0.057000,-0.01760
154,2020-01-24,2020,1,24,2,0.0,.,2020.0,1.0,24.0,0.007130,0.04920,-0.02640,-0.0106,.,-0.06990,0.15900,-0.03270,-0.154000,-0.07230
155,2020-01-24,2020,1,24,4,0.0,.,2020.0,1.0,24.0,-0.009150,0.04490,-0.01260,-0.00749,-0.00918,-0.01670,-0.01190,-0.01160,0.020700,-0.04610
156,2020-01-24,2020,1,24,5,0.0,.,2020.0,1.0,24.0,0.024400,-0.05270,-0.00707,0.0048,-0.00166,-0.01080,0.00842,-0.02800,0.005860,0.00444
157,2020-01-24,2020,1,24,6,0.0,.,2020.0,1.0,24.0,0.004130,-0.00378,0.00813,0.00719,0.0124,0.00905,-0.01150,0.01610,0.020300,-0.00473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7594,2020-06-17,2020,6,17,51,653.0,6.02,2020.0,6.0,17.0,-0.380000,-0.44500,-0.09680,-0.148,0.00955,-0.07610,-0.06580,0.04160,-0.166000,-0.53700
7595,2020-06-17,2020,6,17,53,371.0,4.37,2020.0,6.0,17.0,-0.452000,-0.44700,-0.13100,-0.17,-0.162,-0.08700,-0.24200,0.04320,-0.223000,-0.49600
7596,2020-06-17,2020,6,17,54,133.0,1.46,2020.0,6.0,17.0,-0.243000,-0.55300,0.00479,-0.0685,0.00212,0.01260,0.24000,0.05490,0.072500,-0.36200
7597,2020-06-17,2020,6,17,55,449.0,5.09,2020.0,6.0,17.0,-0.375000,-0.42500,-0.09170,-0.1,-0.123,-0.08550,-0.03900,0.06040,-0.166000,-0.44700


In [11]:
merge2_df = pd.merge(merge1_df, geoids_df, on="statefips", how="left")
del merge2_df['year_y']
del merge2_df['month_y']
del merge2_df['day_y']
merge2_df

,date,year_x,month_x,day_x,statefips,case_rate,new_case_rate,spend_acf,spend_aer,spend_all,spend_all_inchigh,spend_all_inclow,spend_all_incmiddle,spend_apg,spend_grf,spend_hcs,spend_tws,statename,stateabbrev,state_pop2019
0,2020-01-24,2020,1,24,1,0.0,.,-0.000859,0.03580,0.00634,0.0219,0.000657,0.00687,-0.02130,0.00718,0.057000,-0.01760,Alabama,AL,4903185
1,2020-01-24,2020,1,24,2,0.0,.,0.007130,0.04920,-0.02640,-0.0106,.,-0.06990,0.15900,-0.03270,-0.154000,-0.07230,Alaska,AK,731545
2,2020-01-24,2020,1,24,4,0.0,.,-0.009150,0.04490,-0.01260,-0.00749,-0.00918,-0.01670,-0.01190,-0.01160,0.020700,-0.04610,Arizona,AZ,7278717
3,2020-01-24,2020,1,24,5,0.0,.,0.024400,-0.05270,-0.00707,0.0048,-0.00166,-0.01080,0.00842,-0.02800,0.005860,0.00444,Arkansas,AR,3017804
4,2020-01-24,2020,1,24,6,0.0,.,0.004130,-0.00378,0.00813,0.00719,0.0124,0.00905,-0.01150,0.01610,0.020300,-0.00473,California,CA,39512223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7441,2020-06-17,2020,6,17,51,653.0,6.02,-0.380000,-0.44500,-0.09680,-0.148,0.00955,-0.07610,-0.06580,0.04160,-0.166000,-0.53700,Virginia,VA,8535519
7442,2020-06-17,2020,6,17,53,371.0,4.37,-0.452000,-0.44700,-0.13100,-0.17,-0.162,-0.08700,-0.24200,0.04320,-0.223000,-0.49600,Washington,WA,7614893
7443,2020-06-17,2020,6,17,54,133.0,1.46,-0.243000,-0.55300,0.00479,-0.0685,0.00212,0.01260,0.24000,0.05490,0.072500,-0.36200,West Virginia,WV,1792147
7444,2020-06-17,2020,6,17,55,449.0,5.09,-0.375000,-0.42500,-0.09170,-0.1,-0.123,-0.08550,-0.03900,0.06040,-0.166000,-0.44700,Wisconsin,WI,5822434


In [12]:
merge2_df = merge2_df.rename(columns={"year_x": "year", "month_x": "month", "day_x": "day"})
merge2_df

,date,year,month,day,statefips,case_rate,new_case_rate,spend_acf,spend_aer,spend_all,spend_all_inchigh,spend_all_inclow,spend_all_incmiddle,spend_apg,spend_grf,spend_hcs,spend_tws,statename,stateabbrev,state_pop2019
0,2020-01-24,2020,1,24,1,0.0,.,-0.000859,0.03580,0.00634,0.0219,0.000657,0.00687,-0.02130,0.00718,0.057000,-0.01760,Alabama,AL,4903185
1,2020-01-24,2020,1,24,2,0.0,.,0.007130,0.04920,-0.02640,-0.0106,.,-0.06990,0.15900,-0.03270,-0.154000,-0.07230,Alaska,AK,731545
2,2020-01-24,2020,1,24,4,0.0,.,-0.009150,0.04490,-0.01260,-0.00749,-0.00918,-0.01670,-0.01190,-0.01160,0.020700,-0.04610,Arizona,AZ,7278717
3,2020-01-24,2020,1,24,5,0.0,.,0.024400,-0.05270,-0.00707,0.0048,-0.00166,-0.01080,0.00842,-0.02800,0.005860,0.00444,Arkansas,AR,3017804
4,2020-01-24,2020,1,24,6,0.0,.,0.004130,-0.00378,0.00813,0.00719,0.0124,0.00905,-0.01150,0.01610,0.020300,-0.00473,California,CA,39512223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7441,2020-06-17,2020,6,17,51,653.0,6.02,-0.380000,-0.44500,-0.09680,-0.148,0.00955,-0.07610,-0.06580,0.04160,-0.166000,-0.53700,Virginia,VA,8535519
7442,2020-06-17,2020,6,17,53,371.0,4.37,-0.452000,-0.44700,-0.13100,-0.17,-0.162,-0.08700,-0.24200,0.04320,-0.223000,-0.49600,Washington,WA,7614893
7443,2020-06-17,2020,6,17,54,133.0,1.46,-0.243000,-0.55300,0.00479,-0.0685,0.00212,0.01260,0.24000,0.05490,0.072500,-0.36200,West Virginia,WV,1792147
7444,2020-06-17,2020,6,17,55,449.0,5.09,-0.375000,-0.42500,-0.09170,-0.1,-0.123,-0.08550,-0.03900,0.06040,-0.166000,-0.44700,Wisconsin,WI,5822434


## Q2: What is the relationship between unemployment insurance claims and consumer spending trends since the onset of COVID-19 in the United States?

### Data Cleaning

In [24]:
# Part 1: Consumer spending (cs) data
# Check the day of week and select data from Saturday only, the end date for each week used in the unemployment insurance data set

q2_cs_df = merge2_df
q2_cs_df['day_of_week'] = merge2_df['date'].dt.day_name()
q2_cs_df = q2_df.loc[q2_df['day_of_week'] == 'Saturday']
q2_cs_df

,date,year,month,day,statefips,case_rate,new_case_rate,spend_acf,spend_aer,spend_all,...,spend_all_inclow,spend_all_incmiddle,spend_apg,spend_grf,spend_hcs,spend_tws,statename,stateabbrev,state_pop2019,day_of_week
51,2020-01-25,2020,1,25,1,0.00000,.,0.000291,0.0135,0.00580,...,0.00199,0.00612,-0.021100,0.03850,0.0455,-0.02470,Alabama,AL,4903185,Saturday
52,2020-01-25,2020,1,25,2,0.00000,.,-0.064800,0.0268,-0.04460,...,.,-0.05860,0.150000,-0.03790,-0.0852,-0.17400,Alaska,AK,731545,Saturday
53,2020-01-25,2020,1,25,4,0.00000,.,-0.006380,0.0495,-0.01040,...,-0.0101,-0.01140,-0.000813,0.00105,0.0139,-0.02580,Arizona,AZ,7278717,Saturday
54,2020-01-25,2020,1,25,5,0.00000,.,0.030300,-0.0178,0.00584,...,0.0104,0.00174,0.023500,-0.00520,0.0115,0.02110,Arkansas,AR,3017804,Saturday
55,2020-01-25,2020,1,25,6,0.00253,.,-0.003080,-0.0127,0.00565,...,0.00762,0.00773,-0.004620,0.01360,0.0165,-0.00769,California,CA,39512223,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7237,2020-06-13,2020,6,13,51,631.00000,7.48,-0.362000,-0.4060,-0.07740,...,0.0605,-0.05410,-0.062100,0.08880,-0.1660,-0.53600,Virginia,VA,8535519,Saturday
7238,2020-06-13,2020,6,13,53,354.00000,4.04,-0.460000,-0.6000,-0.15900,...,-0.205,-0.11300,-0.305000,0.05920,-0.1670,-0.52200,Washington,WA,7614893,Saturday
7239,2020-06-13,2020,6,13,54,127.00000,1.1,-0.207000,-0.5580,0.02870,...,0.032,0.03750,0.234000,0.07790,-0.1140,-0.35800,West Virginia,WV,1792147,Saturday
7240,2020-06-13,2020,6,13,55,430.00000,10.6,-0.370000,-0.4020,-0.09360,...,-0.105,-0.08520,-0.045500,0.06290,-0.2080,-0.50100,Wisconsin,WI,5822434,Saturday


In [29]:
# Reduce dataframe to Covid case rate and relevant consumer spending data
reduced_q2_cs_df = q2_cs_df.loc[:,['date', 'statename', 'case_rate',
                                  'spend_all', 'spend_acf', 'spend_aer',
                                  'spend_apg', 'spend_grf', 'spend_hcs',
                                  'spend_tws']]
reduced_q2_cs_df

,date,statename,case_rate,spend_all,spend_acf,spend_aer,spend_apg,spend_grf,spend_hcs,spend_tws
51,2020-01-25,Alabama,0.00000,0.00580,0.000291,0.0135,-0.021100,0.03850,0.0455,-0.02470
52,2020-01-25,Alaska,0.00000,-0.04460,-0.064800,0.0268,0.150000,-0.03790,-0.0852,-0.17400
53,2020-01-25,Arizona,0.00000,-0.01040,-0.006380,0.0495,-0.000813,0.00105,0.0139,-0.02580
54,2020-01-25,Arkansas,0.00000,0.00584,0.030300,-0.0178,0.023500,-0.00520,0.0115,0.02110
55,2020-01-25,California,0.00253,0.00565,-0.003080,-0.0127,-0.004620,0.01360,0.0165,-0.00769
...,...,...,...,...,...,...,...,...,...,...
7237,2020-06-13,Virginia,631.00000,-0.07740,-0.362000,-0.4060,-0.062100,0.08880,-0.1660,-0.53600
7238,2020-06-13,Washington,354.00000,-0.15900,-0.460000,-0.6000,-0.305000,0.05920,-0.1670,-0.52200
7239,2020-06-13,West Virginia,127.00000,0.02870,-0.207000,-0.5580,0.234000,0.07790,-0.1140,-0.35800
7240,2020-06-13,Wisconsin,430.00000,-0.09360,-0.370000,-0.4020,-0.045500,0.06290,-0.2080,-0.50100


In [26]:
# Part 2: Unemployment insurance (ui) claims data

#View unemployment insurance dataframe
ui_df

,date,year,month,day,statefips,initial_claims,initial_claims_rate,total_claims,total_claims_rate
0,2020-03-14,2020,3,14,1,1819,0.0811,14889,0.664
1,2020-03-07,2020,3,7,1,1663,0.0742,15320,0.683
2,2020-01-11,2020,1,11,1,3629,0.1620,24772,1.110
3,2020-01-04,2020,1,4,1,4578,0.2040,23101,1.030
4,2020-04-25,2020,4,25,1,74966,3.3400,248821,11.100
...,...,...,...,...,...,...,...,...,...
1219,2020-02-29,2020,2,29,56,471,0.1610,4014,1.370
1220,2020-04-18,2020,4,18,56,4381,1.5000,18683,6.390
1221,2020-02-08,2020,2,8,56,589,0.2020,4096,1.400
1222,2020-05-16,2020,5,16,56,2303,0.7880,19638,6.720


In [15]:
ui_df.dtypes

year                     int64
month                    int64
day_endofweek            int64
statefips                int64
initial_claims           int64
initial_claims_rate    float64
total_claims             int64
total_claims_rate      float64
dtype: object

In [19]:
# Rename column from day_endofweek (Saturdays) to day
ui_df = ui_df.rename(columns={"day_endofweek": "day"})
ui_df

,year,month,day,statefips,initial_claims,initial_claims_rate,total_claims,total_claims_rate
0,2020,3,14,1,1819,0.0811,14889,0.664
1,2020,3,7,1,1663,0.0742,15320,0.683
2,2020,1,11,1,3629,0.1620,24772,1.110
3,2020,1,4,1,4578,0.2040,23101,1.030
4,2020,4,25,1,74966,3.3400,248821,11.100
...,...,...,...,...,...,...,...,...
1219,2020,2,29,56,471,0.1610,4014,1.370
1220,2020,4,18,56,4381,1.5000,18683,6.390
1221,2020,2,8,56,589,0.2020,4096,1.400
1222,2020,5,16,56,2303,0.7880,19638,6.720


In [23]:
# Add date column to ui_df in YYYY-MM-DD format
ui_date = pd.to_datetime(ui_df[['year', 'month', 'day']])
ui_df['date'] = ui_date
ui_df = ui_df[['date'] + [col for col in ui_df.columns if col != 'date']]
ui_df

,date,year,month,day,statefips,initial_claims,initial_claims_rate,total_claims,total_claims_rate
0,2020-03-14,2020,3,14,1,1819,0.0811,14889,0.664
1,2020-03-07,2020,3,7,1,1663,0.0742,15320,0.683
2,2020-01-11,2020,1,11,1,3629,0.1620,24772,1.110
3,2020-01-04,2020,1,4,1,4578,0.2040,23101,1.030
4,2020-04-25,2020,4,25,1,74966,3.3400,248821,11.100
...,...,...,...,...,...,...,...,...,...
1219,2020-02-29,2020,2,29,56,471,0.1610,4014,1.370
1220,2020-04-18,2020,4,18,56,4381,1.5000,18683,6.390
1221,2020-02-08,2020,2,8,56,589,0.2020,4096,1.400
1222,2020-05-16,2020,5,16,56,2303,0.7880,19638,6.720


In [45]:
# Merge state names to ui_df
ui_state_df = pd.merge(ui_df, geoids_df, on='statefips', how='left')

# Drop unneeded columns
ui_state_df = ui_state_df.drop(['statefips','year', 'month', 'day'], axis=1)
ui_state_df

,date,initial_claims,initial_claims_rate,total_claims,total_claims_rate,statename,stateabbrev,state_pop2019
0,2020-03-14,1819,0.0811,14889,0.664,Alabama,AL,4903185
1,2020-03-07,1663,0.0742,15320,0.683,Alabama,AL,4903185
2,2020-01-11,3629,0.1620,24772,1.110,Alabama,AL,4903185
3,2020-01-04,4578,0.2040,23101,1.030,Alabama,AL,4903185
4,2020-04-25,74966,3.3400,248821,11.100,Alabama,AL,4903185
...,...,...,...,...,...,...,...,...
1219,2020-02-29,471,0.1610,4014,1.370,Wyoming,WY,578759
1220,2020-04-18,4381,1.5000,18683,6.390,Wyoming,WY,578759
1221,2020-02-08,589,0.2020,4096,1.400,Wyoming,WY,578759
1222,2020-05-16,2303,0.7880,19638,6.720,Wyoming,WY,578759


In [47]:
# Part 3: Merge consumer spending data with unemployment insurance claims data
cs_ui_df = pd.merge(reduced_q2_cs_df, ui_state_df, how='left', on=['date', 'statename'])
cs_ui_df

,date,statename,case_rate,spend_all,spend_acf,spend_aer,spend_apg,spend_grf,spend_hcs,spend_tws,initial_claims,initial_claims_rate,total_claims,total_claims_rate,stateabbrev,state_pop2019
0,2020-01-25,Alabama,0.00000,0.00580,0.000291,0.0135,-0.021100,0.03850,0.0455,-0.02470,2129,0.0950,20519,0.915,AL,4903185
1,2020-01-25,Alaska,0.00000,-0.04460,-0.064800,0.0268,0.150000,-0.03790,-0.0852,-0.17400,1003,0.2880,10499,3.020,AK,731545
2,2020-01-25,Arizona,0.00000,-0.01040,-0.006380,0.0495,-0.000813,0.00105,0.0139,-0.02580,3475,0.0979,21876,0.616,AZ,7278717
3,2020-01-25,Arkansas,0.00000,0.00584,0.030300,-0.0178,0.023500,-0.00520,0.0115,0.02110,1683,0.1240,14014,1.030,AR,3017804
4,2020-01-25,California,0.00253,0.00565,-0.003080,-0.0127,-0.004620,0.01360,0.0165,-0.00769,45596,0.2350,381865,1.970,CA,39512223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,2020-06-13,Virginia,631.00000,-0.07740,-0.362000,-0.4060,-0.062100,0.08880,-0.1660,-0.53600,27186,0.6160,414079,9.380,VA,8535519
1067,2020-06-13,Washington,354.00000,-0.15900,-0.460000,-0.6000,-0.305000,0.05920,-0.1670,-0.52200,30217,0.7720,440421,11.300,WA,7614893
1068,2020-06-13,West Virginia,127.00000,0.02870,-0.207000,-0.5580,0.234000,0.07790,-0.1140,-0.35800,4327,0.5430,79563,9.980,WV,1792147
1069,2020-06-13,Wisconsin,430.00000,-0.09360,-0.370000,-0.4020,-0.045500,0.06290,-0.2080,-0.50100,25543,0.8230,264071,8.500,WI,5822434


In [48]:
# Part 4: Retrieve data for the top 5 states with most COVID case rates and such data for Florida

# Get data for June 13, the last date with available data
jun13_df = cs_ui_df[cs_ui_df['date'] == '2020-06-13']
jun13_df


,date,statename,case_rate,spend_all,spend_acf,spend_aer,spend_apg,spend_grf,spend_hcs,spend_tws,initial_claims,initial_claims_rate,total_claims,total_claims_rate,stateabbrev,state_pop2019
1020,2020-06-13,Alabama,502.0,-0.01110,-0.205,-0.380,0.0987,0.09050,-0.10000,-0.470,18367,0.819,182349,8.13,AL,4903185
1021,2020-06-13,Alaska,98.7,-0.11800,-0.306,-0.347,-0.0462,0.01020,-0.38500,-0.522,6797,1.950,52589,15.10,AK,731545
1022,2020-06-13,Arizona,478.0,-0.12200,-0.401,-0.496,-0.1700,0.06560,-0.14600,-0.493,22419,0.631,231948,6.53,AZ,7278717
1023,2020-06-13,Arkansas,401.0,0.01050,-0.226,-0.533,0.0534,0.06900,-0.01810,-0.572,9381,0.688,123004,9.03,AR,3017804
1024,2020-06-13,California,381.0,-0.19000,-0.478,-0.646,-0.2710,0.09530,-0.18900,-0.588,241424,1.240,3242837,16.70,CA,39512223
1025,2020-06-13,Colorado,504.0,-0.13300,-0.418,-0.454,-0.1730,0.07500,-0.11900,-0.531,10382,0.330,253167,8.04,CO,5758736
1026,2020-06-13,Connecticut,1262.0,-0.13700,-0.470,-0.548,-0.0948,0.10600,-0.19300,-0.596,11852,0.619,272370,14.20,CT,3565287
1027,2020-06-13,Delaware,1050.0,-0.09200,-0.455,-0.598,-0.0386,0.12400,-0.02770,-0.500,2516,0.516,49631,10.20,DE,973764
1028,2020-06-13,District Of Columbia,1376.0,-0.23500,-0.572,-0.609,-0.1620,0.23500,-0.31300,-0.791,3065,0.748,75999,18.50,DC,705749
1029,2020-06-13,Florida,342.0,-0.05480,-0.242,-0.473,-0.0236,0.06540,-0.04650,-0.479,88148,0.853,1008350,9.75,FL,21477737


In [50]:
# Sort the dataframe to see which 5 states had the highest COVID case rates

jun13_sorted = jun13_df.sort_values('case_rate', ascending=False)
top5 = jun13_sorted.iloc[0:5]
top5_df = pd.DataFrame(top5)
top5_df

,date,statename,case_rate,spend_all,spend_acf,spend_aer,spend_apg,spend_grf,spend_hcs,spend_tws,initial_claims,initial_claims_rate,total_claims,total_claims_rate,stateabbrev,state_pop2019
1052,2020-06-13,New York,1991.0,-0.0970,-0.439,-0.604,-0.195,0.13300,-0.231,-0.515,95150,1.000,1772187,18.6,NY,19453561
1050,2020-06-13,New Jersey,1876.0,-0.0831,-0.432,-0.369,-0.238,0.22100,-0.138,-0.556,26438,0.588,564707,12.6,NJ,8882190
1041,2020-06-13,Massachusetts,1529.0,-0.0854,-0.428,-0.521,-0.043,0.13600,-0.258,-0.565,30411,0.797,608402,15.9,MA,6892503
1059,2020-06-13,Rhode Island,1505.0,-0.2230,-0.556,-0.827,-0.136,-0.00504,-0.113,-0.518,2641,0.475,75724,13.6,RI,1059361
1028,2020-06-13,District Of Columbia,1376.0,-0.2350,-0.572,-0.609,-0.162,0.23500,-0.313,-0.791,3065,0.748,75999,18.5,DC,705749


In [51]:
# Find Florida data
q2_fl_df = jun13_df[jun13_df['statename']=='Florida']
q2_fl_df

,date,statename,case_rate,spend_all,spend_acf,spend_aer,spend_apg,spend_grf,spend_hcs,spend_tws,initial_claims,initial_claims_rate,total_claims,total_claims_rate,stateabbrev,state_pop2019
1029,2020-06-13,Florida,342.0,-0.0548,-0.242,-0.473,-0.0236,0.0654,-0.0465,-0.479,88148,0.853,1008350,9.75,FL,21477737


In [53]:
# Append data of top 5 states and Florida into a single dataframe
q2_top5_fl = top5_df.append(q2_fl_df, sort=False)
q2_top5_fl

,date,statename,case_rate,spend_all,spend_acf,spend_aer,spend_apg,spend_grf,spend_hcs,spend_tws,initial_claims,initial_claims_rate,total_claims,total_claims_rate,stateabbrev,state_pop2019
1052,2020-06-13,New York,1991.0,-0.0970,-0.439,-0.604,-0.1950,0.13300,-0.2310,-0.515,95150,1.000,1772187,18.60,NY,19453561
1050,2020-06-13,New Jersey,1876.0,-0.0831,-0.432,-0.369,-0.2380,0.22100,-0.1380,-0.556,26438,0.588,564707,12.60,NJ,8882190
1041,2020-06-13,Massachusetts,1529.0,-0.0854,-0.428,-0.521,-0.0430,0.13600,-0.2580,-0.565,30411,0.797,608402,15.90,MA,6892503
1059,2020-06-13,Rhode Island,1505.0,-0.2230,-0.556,-0.827,-0.1360,-0.00504,-0.1130,-0.518,2641,0.475,75724,13.60,RI,1059361
1028,2020-06-13,District Of Columbia,1376.0,-0.2350,-0.572,-0.609,-0.1620,0.23500,-0.3130,-0.791,3065,0.748,75999,18.50,DC,705749
1029,2020-06-13,Florida,342.0,-0.0548,-0.242,-0.473,-0.0236,0.06540,-0.0465,-0.479,88148,0.853,1008350,9.75,FL,21477737


### Data Analysis